In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import time
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split

data = pd.read_csv('../input/titanic/train.csv')
data = data.drop(['PassengerId' , 'Name' , 'Ticket' ,'Cabin' ] , axis = 'columns')
data.head()

le =LabelEncoder()
label =le.fit_transform(data['Sex'])
data =data.drop(['Sex'] , axis = 'columns')
data['Sex'] = label

replacer =[]

for i in data['Embarked']:
    if i=='S':
        replacer.append(1)
    else:
        replacer.append(0)

data['Embarked'].replace('S',1)
data =data.drop(['Embarked'] , axis = 'columns')
data['Embarked'] = replacer

for col in data.columns:
    data[col] = data[col].fillna(data[col].mean())
X = data.drop(['Survived'] , axis = 'columns')
Y = data['Survived']
print(X[0:5])
print(Y[0:5])




In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.20, random_state=0)

def get_score(model , X_train , X_test, Y_train , Y_test):
	model.fit(X_train ,Y_train)
	return model.score(X_test ,Y_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , confusion_matrix, classification_report

model = RandomForestClassifier(n_estimators=10)
model.fit(X_train , Y_train)
pred= model.predict(X_test)
print(confusion_matrix(pred ,Y_test))
print(accuracy_score(pred ,Y_test))
print(classification_report(pred ,Y_test))


In [ ]:
#parameter tuning

from sklearn.model_selection import RandomizedSearchCV
n_estimators  = [int(x) for x in np.linspace(start =200 ,stop = 2000 ,num=10)]
max_features =['auto','sqrt', 'log2'] 
max_depth = [int(x) for x in np.linspace(10,1000,10)]
min_sample_split = [1,3,4,5,7,9]
min_samples_leaf=[1,2,4,6,8]
criterion= ['entropy' , 'gini']

random_grid={
    'n_estimators' : n_estimators ,
    'max_features' : max_features ,
    'max_depth' : max_depth ,
    'min_samples_split' : min_sample_split ,
    'min_samples_leaf' : min_samples_leaf ,
    'criterion':criterion 
}

 

In [ ]:
model = RandomForestClassifier()
model_random_cv = RandomizedSearchCV(estimator=model ,   param_distributions=random_grid , n_iter=10 ,
                                    cv =3 , verbose=2, random_state=100 ,n_jobs=-1)

model_random_cv.fit(X_train, Y_train)

In [ ]:
best_random_grid = model_random_cv.best_estimator_
print(get_score(model ,X_train, X_test, Y_train, Y_test))
print(get_score(best_random_grid ,X_train, X_test, Y_train, Y_test))
model_random_cv.best_params_

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'criterion':[model_random_cv.best_params_['criterion']] , 
    'max_depth':[model_random_cv.best_params_['max_depth'] ], 
    'max_features':[model_random_cv.best_params_['max_features']] , 
    'min_samples_leaf':[model_random_cv.best_params_['min_samples_leaf'] ,
                       model_random_cv.best_params_['min_samples_leaf']+2,
                       model_random_cv.best_params_['min_samples_leaf'] +4]  , 
    'min_samples_split':[model_random_cv.best_params_['min_samples_split']-2 ,
                       model_random_cv.best_params_['min_samples_split']-1,
                       model_random_cv.best_params_['min_samples_split']  ,
                        model_random_cv.best_params_['min_samples_split'] +1,
                        model_random_cv.best_params_['min_samples_split'] +2]  ,
    'n_estimators':[model_random_cv.best_params_['n_estimators']-200 ,
                       model_random_cv.best_params_['n_estimators']-100,
                       model_random_cv.best_params_['n_estimators']  ,
                        model_random_cv.best_params_['n_estimators'] +100,
                        model_random_cv.best_params_['n_estimators'] +200] 
    
}


param_grid
 


In [ ]:
model = RandomForestClassifier()
model_random_cv = GridSearchCV(estimator=model ,   param_grid=param_grid   ,
                                    cv =3 ,verbose=2 , n_jobs=-1)

model_random_cv.fit(X_train, Y_train)

In [ ]:
best_random_grid = model_random_cv.best_estimator_
print(get_score(model ,X_train, X_test, Y_train, Y_test))
print(get_score(best_random_grid ,X_train, X_test, Y_train, Y_test))


In [ ]:
from hyperopt import hp